# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [1]:
import ib_insync
print(ib_insync.__all__)

['util', 'Object', 'ContractDetails', 'ContractDescription', 'ComboLeg', 'UnderComp', 'DeltaNeutralContract', 'OrderComboLeg', 'OrderState', 'ScannerSubscription', 'SoftDollarTier', 'Execution', 'CommissionReport', 'ExecutionFilter', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'TickAttrib', 'NewsProvider', 'DepthMktDataDescription', 'PnL', 'PnLSingle', 'AccountValue', 'RealTimeBar', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'MktDepthData', 'DOMLevel', 'BracketOrder', 'TradeLogEntry', 'ScanData', 'TagValue', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'OrderCondition', 'ExecutionCondition', 'OperatorCondition', 'MarginCondition', 'ContractCondition', 'TimeCondition', 'PriceCondition', 'PercentChangeCondition', 'VolumeCondition',

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_insync import *
util.startLoop()

*startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application (remember it must have an open API port):

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=111230)

<IB connected to 127.0.0.1:7497 clientId=111230>

If the connection failed, then double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [4]:
ib.positions()

[Position(account='DU946735', contract=Contract(secType='CASH', conId=12087792, symbol='EUR', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), position=186000.0, avgCost=1.2182257),
 Position(account='DU946735', contract=Contract(secType='CFD', conId=111987422, symbol='IBDE30', currency='EUR', localSymbol='IBDE30', tradingClass='IBDE30'), position=25.0, avgCost=11655.669984)]

Or filter the account values to get the liquidation value:

In [5]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU946735', tag='NetLiquidationByCurrency', value='959718.1626', currency='BASE', modelCode='')]

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting it's attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [6]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [7]:
%%time

contract = Stock('TSLA', 'SMART', 'USD')
cds = ib.reqContractDetails(contract)

CPU times: user 111 ms, sys: 2.54 ms, total: 114 ms
Wall time: 675 ms


Requesting contract details generally takes about 100 - 300 ms. Let's have a look at the actual
contract details that were sent:

In [8]:
cds

[ContractDetails(marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,EDGX,FOXRIVER,NYSENAT,PSX', priceMagnifier=1, longName='TESLA INC', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST5EDT', tradingHours='20190927:0400-20190927:2000;20190928:CLOSED;20190929:CLOSED;20190930:0400-20190930:2000;20191001:0400-20191001:2000;20191002:0400-20191002:2000;20191003:0400-20191003:2000;20191004:0400-20191004:2000;20191005:CLOSED;20191006:CLOSED;20191

### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [9]:
util.logToConsole()

To see all debug messages (including network traffic):

In [10]:
import logging
util.logToConsole(logging.DEBUG)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [11]:
ib.disconnect()

2019-09-28 03:15:36,790 ibapi.client DEBUG 140285400770488 isConn: 2, connConnected: True
2019-09-28 03:15:36,791 ib_insync.ib INFO Disconnecting from 127.0.0.1:7497, 148 B sent in 7 messages, 19.1 kB received in 360 messages, session time 33.9 s.
2019-09-28 03:15:36,792 ibapi.client DEBUG 140285400770488 connState: 2 -> 0
2019-09-28 03:15:36,792 ibapi.client INFO disconnecting
2019-09-28 03:15:36,793 ibapi.wrapper INFO ANSWER connectionClosed {}
2019-09-28 03:15:36,793 ibapi.client DEBUG 140285400770488 connState: None -> 0
2019-09-28 03:15:36,796 ibapi.client DEBUG 140285400770488 isConn: 0, connConnected: None
2019-09-28 03:15:36,797 ib_insync.client INFO Disconnected
2019-09-28 03:15:36,798 ibapi.client DEBUG 140285400770488 connState: None -> 0
